In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib as plt
import seaborn as sns

df = pd.read_csv("train.csv")
df

# How do you correlate passenger class with survival chances?

In [ ]:
pd.crosstab(df.Survived, df.Pclass, margins=True)


In [ ]:
temp4 = pd.crosstab(df['Pclass'], df['Survived']) 
temp4.plot(kind='bar')
plt.show()


# What is the percenbtage of survivors in each passenger class, therefore, how did the passenger class influence survivability? 

In [ ]:
# 62% dos class 1 sobreviveram
# 48% dos class 2 sobreviveram
# 25% dos class 3 sobreviveram
temp1 = df['Pclass'].value_counts(ascending=False) 

temp2 = df.pivot_table(values='Survived',index=['Pclass'],aggfunc=lambda x: x.mean()) 

print ('Number of passengers in each class:') 
print (temp1) 

print ('Percentage of survivals in each class:') 
print (temp2 )


temp2.plot(kind='bar')
plt.show()


# How did the sex influence survival 

In [ ]:
pd.crosstab(([df['Pclass'],df['Sex']]),df['Survived'])

# Some simple transformation...

In [ ]:
tempx = pd.crosstab(([df['Pclass'],df['Sex']]),df['Survived'].where(df['Survived']==1)) 

tempx.plot(kind='bar', color=['gold','blue'], grid=False)
plt.show()

In [ ]:
def status(feature): 
	print ('Processing',feature,': ok' )

def process_family(): 
	# introducing a new feature : the size of families (including the passenger) 
	df['FamilySize'] = df['Parch'] + df['SibSp'] + 1 

	# introducing other features based on the family size 
	df['family'] = df['SibSp'].map(lambda s : 1 if s > 1 else 0) 
	
	status('family')


In [ ]:
process_family()

In [ ]:
df

# What are the age groups with more and less survivability? 

In [ ]:
group_by_age = pd.cut(df["Age"], [0,18,25,40,60,100]) 

age_grouping = df.groupby(group_by_age)['Survived'].count() 

(age_grouping/age_grouping.sum()).plot(kind='pie',autopct='%.1f')

plt.show()

In [ ]:
group_by_age = pd.cut(df["Age"], [0,18,25,40,60,100]) 

age_grouping = df.groupby(group_by_age)['Survived'].sum() 

(age_grouping/age_grouping.sum()).plot(kind='pie',autopct='%.1f')

plt.show()

In [ ]:
da = df.drop('Ticket', 1)

da = da.drop('Cabin', 1)

da = da.dropna()

da.count()


# How good is the following logistic regression model? Look at the R2 and p-values and conclude regarding both the quality of the model and the significance of each variable 

In [ ]:
import patsy 

df_train=da.iloc[0:600,:] 
df_test = da.iloc[0:112,:] 
formula = 'Survived ~ C(Pclass) + C(Sex) + Age + SibSp + C(Embarked) + Parch' 

y_train,x_train = patsy.dmatrices(formula, data=df_train,return_type='dataframe') 

y_test,x_test = patsy.dmatrices(formula, data=df_test,return_type='dataframe')


In [ ]:
import statsmodels.api as sm

model = sm.Logit(y_train,x_train) 

res = model.fit() 

res.summary()



# A few examples of estimation versus real value... What is the estimation and what is the real value?

In [ ]:
model = sm.Logit(y_test,x_test) 

res = model.fit() 

y_pred = res.predict(x_test) 

np.stack((y_test['Survived'],y_pred),1)


# What does precision and recall for each case (survived/dd not survive) tell us?

In [ ]:
y_pred_flag=y_pred>0.7 

print(pd.crosstab(y_train.Survived,y_pred_flag))

import sklearn.metrics as smet 

print( smet.classification_report(y_test,y_pred_flag)  )


# Can you do better than this by choosing carefully the variables to use? What would you try? Try it once....

# THE END